## Cookie Cats Mobile Game Dataset

- userid: user id records, total 90189 count
- version: ‘gate_30’ and 'gate_40', work as control group and test group
- sum_gamerounds: total game rounds for each user
- retention_1: whether the user stays after 1 day
- retention_7: whether the user stays after 7 day

In [2]:
import kagglehub
import os
import pandas as pd
import numpy as np

# Download latest version
path = kagglehub.dataset_download("yufengsui/mobile-games-ab-testing")

print("Path to dataset files:", path)
print(os.listdir(path))

/Users/edwardrenaldi/Library/Python/3.9/lib/python/site-packages/urllib3/__init__.py:35: NotOpenSSLWarning: urllib3 v2 only supports OpenSSL 1.1.1+, currently the 'ssl' module is compiled with 'LibreSSL 2.8.3'. See: https://github.com/urllib3/urllib3/issues/3020
  warnings.warn(
/Users/edwardrenaldi/Library/Python/3.9/lib/python/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


100%|██████████| 490k/490k [00:01<00:00, 497kB/s]

Extracting files...
Path to dataset files: /Users/edwardrenaldi/.cache/kagglehub/datasets/yufengsui/mobile-games-ab-testing/versions/1
['cookie_cats.csv']


In [3]:
df = pd.read_csv(os.path.join(path, 'cookie_cats.csv'))
df.head()

,userid,version,sum_gamerounds,retention_1,retention_7
0,116,gate_30,3,False,False
1,337,gate_30,38,True,False
2,377,gate_40,165,True,False
3,483,gate_40,1,False,False
4,488,gate_40,179,True,True


In [4]:
# Check missing values
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 90189 entries, 0 to 90188
Data columns (total 5 columns):
 #   Column          Non-Null Count  Dtype 
---  ------          --------------  ----- 
 0   userid          90189 non-null  int64 
 1   version         90189 non-null  object
 2   sum_gamerounds  90189 non-null  int64 
 3   retention_1     90189 non-null  bool  
 4   retention_7     90189 non-null  bool  
dtypes: bool(2), int64(2), object(1)
memory usage: 2.2+ MB


In [5]:
df.isnull().sum()

userid            0
version           0
sum_gamerounds    0
retention_1       0
retention_7       0
dtype: int64

# **Let's start the EDA**
## Statistics about gamerounds

In [6]:
df['sum_gamerounds'].describe([0, 0.1, 0.2, 0.3, 0.4, 0.5, 0.6, 0.7, 0.8, 0.9, 0.95, 0.99, 1])

count    90189.000000
mean        51.872457
std        195.050858
min          0.000000
0%           0.000000
10%          1.000000
20%          3.000000
30%          6.000000
40%         11.000000
50%         16.000000
60%         25.000000
70%         40.000000
80%         67.000000
90%        134.000000
95%        221.000000
99%        493.000000
100%     49854.000000
max      49854.000000
Name: sum_gamerounds, dtype: float64

| Metric                       | Meaning                                                                                               |
| ---------------------------- | ----------------------------------------------------------------------------------------------------- |
| **count**                    | Number of data points: **90,189** users.                                                              |
| **mean**                     | Average number of rounds played: **51.87**.                                                           |
| **std**                      | Standard deviation (spread): **195.05** → Very large, which hints at some extreme values or outliers. |
| **min / 0%**                 | Minimum rounds played: **0** — Some users didn't play at all.                                         |
| **10%**                      | 10% of the players played <= 1 round                                                                     |
| **20%**                      | 20% of the players played <= 3 rounds                                                                    |
| **30%**                      | 30% of the players played <= 6 rounds                                                                    |
| **50%**                      | 50% of the players played <= 16 rounds                                                                   |
| **70%**                      | 70% of the players played <= 40 rounds                                                                   |
| **90%**                      | 90% of the players played <= 134 rounds                                                                   |
| **95%**                      | 95% of the players played <= 221 rounds                                                                   |
| **99%**                      | 99% of the players played <= 493 rounds                                                                   |
| **Maximum / 100%**                      | All of the players played <= 49854 rounds (which is weird in my opinion)                                                          |


In [7]:
df.sort_values(by = 'sum_gamerounds', ascending = False)

,userid,version,sum_gamerounds,retention_1,retention_7
57702,6390605,gate_30,49854,False,True
7912,871500,gate_30,2961,True,True
29417,3271615,gate_40,2640,True,False
43671,4832608,gate_30,2438,True,True
48188,5346171,gate_40,2294,True,True
...,...,...,...,...,...
59687,6610805,gate_30,0,False,False
23707,2639282,gate_40,0,False,False
59670,6608718,gate_30,0,False,False
59645,6604544,gate_30,0,False,False


### After checking the values, we can conclude that there are outliers in the data. Since it may distort the typical behaviors of a user, we need to transform the data.
### This time, I'm using a log transformation

In [8]:
log_values = np.log1p(df['sum_gamerounds'])  #log(1 + x)
mean = log_values.mean()
std = log_values.std()

df['log_sum_gamerounds'] = log_values
df['log_sum_gamerounds'].describe([0, 0.1, 0.2, 0.3, 0.4, 0.5, 0.6, 0.7, 0.8, 0.9, 0.95, 0.99, 1])


count    90189.000000
mean         2.879746
std          1.503792
min          0.000000
0%           0.000000
10%          0.693147
20%          1.386294
30%          1.945910
40%          2.484907
50%          2.833213
60%          3.258097
70%          3.713572
80%          4.219508
90%          4.905275
95%          5.402677
99%          6.202536
100%        10.816874
max         10.816874
Name: log_sum_gamerounds, dtype: float64

### This data is using retention 1 and retention 7 to make statistical inferences about rentention rate in this game
### Therefore, let's make a new column where **Retention is True** when players come back after 1 day and 7 days of playing

In [9]:
df['Retention'] = df.apply(lambda row: True if (row['retention_1'] == True) and (row['retention_7'] == True) else False, axis = 1)
df['Group'] = df.apply(lambda row: 'Group A' if (row['version'] == 'gate_30') else 'Group B', axis = 1)

In [23]:
print(df['retention_1'].value_counts())
print(df['retention_7'].value_counts())
print(df['Retention'].value_counts())
print("---")

print(f"Day 1 Retention Rate: {round(df['retention_1'].mean() * 100, 2)}%")
print(f"Day 7 Retention Rate: {round(df['retention_7'].mean() * 100, 2)}%")
print(f"Retention Rate: {round(df['Retention'].mean() * 100, 2)}%")


retention_1
False    50036
True     40153
Name: count, dtype: int64
retention_7
False    73408
True     16781
Name: count, dtype: int64
Retention
False    77007
True     13182
Name: count, dtype: int64
---
Day 1 Retention Rate: 44.52%
Day 7 Retention Rate: 18.61%
Retention Rate: 14.62%


In [25]:
df.columns

Index(['userid', 'version', 'sum_gamerounds', 'retention_1', 'retention_7',
       'log_sum_gamerounds', 'Retention', 'Group'],
      dtype='object')

In [28]:
df.groupby(['Retention'])['sum_gamerounds'].describe()

,count,mean,std,min,25%,50%,75%,max
Retention,,,,,,,,
False,77007.0,28.734050,185.981727,0.0,4.0,12.0,33.0,49854.0
True,13182.0,187.043089,191.926715,0.0,67.0,130.0,240.0,2961.0
